# NOTE

For Spark 3 Cosmos DB connector has slightly different configuration. Please select different snippest if a Spark 3.1 + pool is attached.

In [1]:
# Load a streaming Spark DataFrame from a Cosmos DB container
# To select a preferred list of regions in a multi-region Cosmos DB account, add .option("spark.cosmos.preferredRegions", "<Region1>,<Region2>")

# For Spark 2.4
# dfStream = spark.readStream\
#     .format("cosmos.oltp")\
#     .option("spark.synapse.linkedService", "CosmosDbNoSql1")\
#     .option("spark.cosmos.container", "orders")\
#     .option("spark.cosmos.changeFeed.readEnabled", "true")\
#     .option("spark.cosmos.changeFeed.startFromTheBeginning", "true")\
#     .option("spark.cosmos.changeFeed.checkpointLocation", "/localReadCheckpointFolder")\
#     .option("spark.cosmos.changeFeed.queryName", "streamQuery")\
#     .load()

# For Spark 3.1 +
dfStream = spark.readStream\
    .format("cosmos.oltp.changeFeed")\
    .option("spark.synapse.linkedService", "CosmosDbNoSql1")\
    .option("spark.cosmos.container", "orders")\
    .option("spark.cosmos.changeFeed.startFrom", "Beginning")\
    .option("spark.cosmos.changeFeed.mode", "Incremental")\
    .load()

In [2]:
from   pyspark.sql.functions import * 

In [3]:
dfStream

In [4]:
dfStream=dfStream.withColumn("Order_Timestamp",to_timestamp("Order_Timestamp"))

In [6]:
dfStream.writeStream.outputMode("append").format("delta").option("path","abfss://streamsynapse@synapasestream.dfs.core.windows.net/synapse/workspaces/streamoutput/").option("checkpointLocation","abfss://streamsynapse@synapasestream.dfs.core.windows.net/synapse/workspaces/checkpoint/").start()

In [8]:
display(spark.read.load('abfss://streamsynapse@synapasestream.dfs.core.windows.net/synapse/workspaces/streamoutput/'))